<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/elegiveis_cb_fees/elegiveis_cb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd

In [2]:
!pip install dask[dataframe] pyxlsb

In [3]:
#especificar nome dos ficheiros

resumo_user_name="P_Resumo_users_setembro23.xlsb"
apoio_name="Apoio_Base_Proactiva.xlsb"

In [4]:
#importar ficheiros - importar resumo_user e apoio

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()

Saving Apoio_Base_Proactiva.xlsb to Apoio_Base_Proactiva.xlsb


Saving P_Resumo_users_setembro23.xlsb to P_Resumo_users_setembro23.xlsb


In [72]:
#ler dataframes
resumo_user=pd.read_excel(resumo_user_name, sheet_name=0)

apoio_base_sr=pd.read_excel(apoio_name, sheet_name="SR")
apoio_base_ativacao=pd.read_excel(apoio_name, sheet_name="Ativação")
apoio_base_gal=pd.read_excel(apoio_name, sheet_name="GAL-1")
apoio_base_portabilidade=pd.read_excel(apoio_name, sheet_name="Portabilidades")

#substituir nomes das colunas no ficheiro resumo_user
resumo_user2=resumo_user.copy()

resumo_user2.columns=[i.replace("RRR", "").strip() if i.find("RRR")!=-1 else i for i in resumo_user.columns.tolist()]

In [25]:
apoio_base_ativacao2=apoio_base_ativacao.copy()



Retention Username
DVMONTEIRO            36
MAANSOARES            17
CPMEIRELES            16
JARISANTOS            15
DJLEITE               14
                      ..
MACONASCIMEN           1
CMARINEVES             1
KAREIS                 1
LIANSANTOS             1
FRSOCARVALHO           1
Name: count, Length: 119, dtype: int64

In [10]:
resumo_user2.head(2)

,Service Request ID,Retention UserName,Eligivel SR(P),Eligivel SR(V),Eligivel User C/Ajustes>0,Eligivel User_IB>0,Eligivel CB C/Ajustes (SR),Elegivel IB (SR),Valor User,Linhas CB a Zero,Valor User IB,Linhas CB a Zero IB
0,1-43938294078,ANARITASANTO,Y,Y,Y,Y,Y,Y,623.57,N,803.36,N
1,1-43804238364,RERIRIBEIRO,Y,Y,Y,Y,Y,Y,479.40,N,488.97,N


In [12]:
apoio_base_ativacao.head(2)

,Cycle ID,Cycle Name,Incentive Type ID,Incentive Type Name,Scheme Name,Channel IN,Commissioned SFID,Commissioned Sfid Name,Event Sfid,Retention Username,...,Justified Absences,Unjustified Absences,Invoice Payment Days,Chargeback Rate,Original Event Type Name,Sr Chargeback Nr Points,Value Pre WithHolding,Original Value,Parceiro,Elegivel
0,5056,Ciclo RPR Setembro-2023,adacfd93-e04d-44ee-9645-27ecc33672be,RPR_UPFRONT NOS,RPR_UPFRONT_PROATIVA_RESIDENCIAL,PROACTIVA,D.S05.PO1.189,Ana Santos,D.S05.PO1.189,ANARITASANTO,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,D.S05,NaN
1,5056,Ciclo RPR Setembro-2023,adacfd93-e04d-44ee-9645-27ecc33672be,RPR_UPFRONT NOS,RPR_UPFRONT_PROATIVA_RESIDENCIAL,PROACTIVA,D.S05.PO1.123,Renato Ribeiro,D.S05.PO1.123,RERIRIBEIRO,...,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,D.S05,NaN


In [116]:
#sr -> coluna h (Elegivel IB (SR)) e l (Linhas CB a Zero IB)
#activação -> coluna k (Valor User IB) e coluna "service request id" do ficheiro gal n+1
#portabilidade -> coluna k (Valor User IB)

lista1=["Service Request ID", "Service Request ID", "Retention Username", "Purchase Order Item", "Retention Username"]
lista2=["Elegivel IB (SR)", "Linhas CB a Zero IB", "Valor User IB", "Service Request ID", "Valor User IB"]

for count, (i, j) in enumerate(zip(lista1, lista2)):

  if count==0:
    apoio_base_sr["Elegivel"]=apoio_base_sr[[i]].merge(right=resumo_user2, how="left", on=i) [j]

  elif count==1:
    apoio_base_sr["Valor Zero"]=apoio_base_sr[[i]].merge(right=resumo_user2, how="left", on=i) [j]


  elif count==2:
    #elegivel
    for user in apoio_base_ativacao[i].unique().tolist():
      indice=apoio_base_ativacao.loc[apoio_base_ativacao[i]==user, i].index.tolist()
      valor=resumo_user2.loc[resumo_user2["Retention UserName"]==user, j].unique().tolist()[0]
      if valor>0:
        apoio_base_ativacao.loc[indice, "Elegivel"]="s"
      else:
        apoio_base_ativacao.loc[indice, "Elegivel"]="n"


  elif count==3:
    for order in apoio_base_ativacao[i].unique().tolist():
      indice=apoio_base_ativacao.loc[apoio_base_ativacao[i]==order, i].index.tolist()

      valor=apoio_base_gal.loc[apoio_base_gal[i]==order, j].unique().tolist()[0]
      apoio_base_ativacao.loc[indice, "gal-1"]=valor
    #apoio_base_ativacao["Elegivel"]=apoio_base_ativacao[[i]].merge(right=apoio_base_gal, how="left", on=i) [j]

  #else:
    #apoio_base_portabilidade["Elegivel"]=apoio_base_portabilidade.merge(right=resumo_user2, how="left", left_on=i, right_on="Retention UserName") [j]


IndexError: list index out of range

In [115]:
apoio_base_ativacao["Elegivel"].unique()

array(['s', 'n'], dtype=object)